# Where have I been?
[WHIB](http://www.bleatinc.com/) is constantly running on my iPhone and tracks my location with minimal battery impact.
This notebook here parses and display the exported data[^1] from the app into something pretty.

[^1]: A 2$/month *premium* feature lets you export all the data as CSV, which is easy to parse and display.

In [1]:
import os
import glob
import pandas
from datetime import datetime
import geopy
import geopy.distance
import geopy.geocoders
from geopy.extra.rate_limiter import RateLimiter
import folium
import folium.plugins

In [2]:
# Which year do we want to look at?
whichyear = 2022

In [3]:
# Tile providers
# Available tiles: https://github.com/python-visualization/folium/blob/master/folium/folium.py#L75
#tileprovider = 'Mapbox Bright'
tileprovider = 'Cartodb positron'
#tileprovider = 'Cartodb dark_matter'
#tileprovider = 'Mapbox Control Room'
#tileprovider = 'Stamen Toner'

In [4]:
# Default zoom level and circle marker radius for map
zoom_start = 4
radius = 10

In [5]:
# Settings for address lookup
# Set an unique user agent
geopy.geocoders.options.default_user_agent = 'Jahresrückblick Habi. Contact habi@gna.ch if you have an issue with me!'
# Be patient
geolocator = geopy.geocoders.Nominatim(timeout=5)
# Don't be too needy
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [6]:
# Read in locations from newest CSV-file in the current directory
file = sorted(glob.glob('journey*.csv'),key=os.path.getmtime)[-1]
print('I am reading %s' % file)
locations = pandas.read_csv(file)

I am reading journey.2023.01.17.csv


In [7]:
# Modify the dataframe
locations.drop(['Crumb'], axis=1, inplace=True)
locations.rename(columns={'LocalDate': 'Date'}, inplace=True)
locations.rename(columns={'LocalTime': 'Time'}, inplace=True)
locations.rename(columns={'Altitude (in metres)': 'Altitude'}, inplace=True)
locations.rename(columns={'Accuracy (in metres)': 'Accuray'}, inplace=True)

In [8]:
# Make us a proper date column, based on https://stackoverflow.com/a/26763793
locations['Date'] = pandas.to_datetime(locations['Date'])
# Make us a year, month and weekday colum, based on https://stackoverflow.com/q/48623332
locations['Year'] = locations.Date.dt.year
locations['Month'] = locations.Date.dt.month
locations['Day'] = locations.Date.dt.day
locations['Weekday'] = locations.Date.dt.dayofweek

In [9]:
len(locations)

86053

In [10]:
# Drop all values not in 'whichyear'
# https://stackoverflow.com/a/27360130
locations.drop(locations[locations.Year != whichyear].index, inplace=True)
# Reset index, so that we can find the correct date lateron
locations.reset_index(drop=True, inplace=True)

In [11]:
len(locations)

23500

In [12]:
start = datetime(day=1, month=1, year=whichyear)
finish = datetime(day=1, month=1, year=whichyear+1)
time = finish - start
print('%s had %s days' % (whichyear, time.days))
print('Which is %s hours' % (time.days * 24))
print('Assuming 8 hours of sleep (phone off), we then have %0.4s locations per hour' % (len(locations) / (time.days * 16)))

2022 had 365 days
Which is 8760 hours
Assuming 8 hours of sleep (phone off), we then have 4.02 locations per hour


In [13]:
# Drop all values not in a certain month
# locations.drop(locations[locations.Month != 7].index, inplace=True)

In [14]:
locations.head()

,Date,Time,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
0,2022-01-01,15:16,46.935317,7.417864,558,25,2022,1,1,5
1,2022-01-01,15:16,46.933820,7.420402,558,5,2022,1,1,5
2,2022-01-01,15:21,46.927661,7.427314,602,5,2022,1,1,5
3,2022-01-01,15:26,46.919270,7.426294,650,5,2022,1,1,5
4,2022-01-01,15:27,46.918811,7.431872,741,107,2022,1,1,5


In [15]:
locations.tail()

,Date,Time,Latitude,Longitude,Altitude,Accuray,Year,Month,Day,Weekday
23495,2022-12-31,18:46,46.937855,7.416633,562,11,2022,12,31,5
23496,2022-12-31,18:47,46.937927,7.416503,553,676,2022,12,31,5
23497,2022-12-31,18:49,46.935573,7.418067,0,0,2022,12,31,5
23498,2022-12-31,18:49,46.935573,7.418067,0,0,2022,12,31,5
23499,2022-12-31,18:49,46.935859,7.417023,0,65,2022,12,31,5


In [16]:
# In 2022, there is one datapoint in Croatia on August 31
# I have no idea how that happened, as I'm very sure I've been at home there :)
# Let's just drop this *one* point
print(locations.Longitude.max())
locations = locations.drop(locations[(locations['Longitude'] == 14.84985) & (locations['Date'] == '2022-08-31')].index)
print(locations.Longitude.max())

14.84985
11.633607


In [17]:
# Show the extreme locations on a map
# Marker colors from here: https://stackoverflow.com/a/41993318
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
# Altitude
for c, loc in locations.sort_values(by=['Altitude'],
                                    ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='Highest: %s' % (loc.Date),
                       ).add_to(m)
# North
for c, loc in locations.sort_values(by=['Latitude'],
                                    ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='North: %s' % (loc.Date),
                       ).add_to(m)
# East
for c, loc in locations.sort_values(by=['Longitude'],
                                    ascending=False).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='East: %s' % (loc.Date),
                       ).add_to(m)
# South
for c, loc in locations.sort_values(by=['Latitude']).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='South: %s' % (loc.Date),
                       ).add_to(m)
# West
for c, loc in locations.sort_values(by=['Longitude']).head(1).iterrows():
    folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                        radius=radius,
                        popup='West: %s' % (loc.Date),
                       ).add_to(m)
m

In [18]:
# Show north, east, south, west extreme (with averaged lat/lon for each)
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
                tiles=tileprovider,
                zoom_start=zoom_start)
folium.CircleMarker(location=[locations.Latitude.mean(),
                              locations.Longitude.min()],
                    radius=radius,
                    popup='Average West').add_to(m)
folium.CircleMarker(location=[locations.Latitude.mean(),
                              locations.Longitude.max()],
                    radius=radius,
                    popup='Average East').add_to(m)
folium.CircleMarker(location=[locations.Latitude.max(),
                              locations.Longitude.mean()],
                    radius=radius,
                    popup='Average North').add_to(m)
folium.CircleMarker(location=[locations.Latitude.min(),
                              locations.Longitude.mean()],
                    radius=radius,
                    popup='Average South').add_to(m)
m

In [19]:
# How far did we come?
print('In %s, we traveled %0.0f km north-south' % (whichyear,
                                                   geopy.distance.geodesic([locations.Latitude.min(),
                                                                            locations.Longitude.mean()],
                                                                         [locations.Latitude.max(),
                                                                          locations.Longitude.mean()]).km))
print('In %s, we traveled %0.0f km east-west' % (whichyear,
                                                 geopy.distance.geodesic([locations.Latitude.mean(),
                                                                          locations.Longitude.min()],
                                                                         [locations.Latitude.mean(),
                                                                          locations.Longitude.max()]).km))


In 2022, we traveled 826 km north-south
In 2022, we traveled 389 km east-west


By using [the geocoding library for Python](https://github.com/geopy/geopy) and the [OpenStreetMap reverse geocoding tool](https://wiki.openstreetmap.org/wiki/Nominatim) we can assign addresses to locations.

In [20]:
# Get all extrema locations
location_average = [locations['Latitude'].mean(), locations['Longitude'].mean()]
location_median = [locations['Latitude'].median(), locations['Longitude'].median()]
location_north = [locations[locations.Latitude == locations.Latitude.max()].Latitude.values[0],
                  locations[locations.Latitude == locations.Latitude.max()].Longitude.values[0]]
location_east = [locations[locations.Longitude == locations.Longitude.max()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.max()].Longitude.values[0]]
location_south = [locations[locations.Latitude == locations.Latitude.min()].Latitude.values[0],
                  locations[locations.Latitude == locations.Latitude.min()].Longitude.values[0]]
location_west = [locations[locations.Longitude == locations.Longitude.min()].Latitude.values[0],
                 locations[locations.Longitude == locations.Longitude.min()].Longitude.values[0]]
location_top = [locations[locations['Altitude'] == locations['Altitude'].max()].Latitude.values[0],
                locations[locations['Altitude'] == locations['Altitude'].max()].Longitude.values[0]]

In [21]:
# Turn debug on to print and thus find the right value of the address
debug = True

In [22]:
# According to https://github.com/openstreetmap/Nominatim/issues/885#issuecomment-358123829
# we have to
# > Look for the first of 'city', 'town', 'village', 'hamlet', 'suburb'
# Let's do this!
potentialplacename = ['city', 'town', 'village', 'hamlet', 'county', 'municipality', 'suburb']

In [23]:
# Address lookup with `geopy` and https://wiki.openstreetmap.org/wiki/Nominatim
location_average_geo = geolocator.reverse(location_average)
# Print the details, so we can get the correct value to save below
if debug:
    print(location_average_geo.raw)
# Save us a name and print it
for p in potentialplacename:
    if location_average_geo.raw.get('address').get(p):
        name_average = location_average_geo.raw.get('address').get(p)
print('The average location in %s was in %s' % (whichyear, name_average))
# Show the point on a map
m = folium.Map(location=[float(location_average_geo.raw.get('lat')),
                         float(location_average_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_average_geo.raw.get('lat')),
                              float(location_average_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_average).add_to(m)
m

{'place_id': 15181978, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 1642782692, 'lat': '46.6042276', 'lon': '7.7488532', 'display_name': 'Rengg, Reichenbach im Kandertal, Verwaltungskreis Frutigen-Niedersimmental, Verwaltungsregion Oberland, Bern/Berne, 3723, Schweiz/Suisse/Svizzera/Svizra', 'address': {'locality': 'Rengg', 'city': 'Reichenbach im Kandertal', 'county': 'Verwaltungskreis Frutigen-Niedersimmental', 'state_district': 'Verwaltungsregion Oberland', 'state': 'Bern/Berne', 'ISO3166-2-lvl4': 'CH-BE', 'postcode': '3723', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['46.5942276', '46.6142276', '7.7388532', '7.7588532']}
The average location in 2022 was in Verwaltungskreis Frutigen-Niedersimmental


In [24]:
location_median_geo = geolocator.reverse(location_median)
if debug:
    print(location_median_geo.raw)
for p in potentialplacename:
    if location_median_geo.raw.get('address').get(p):
        name_median = location_median_geo.raw.get('address').get(p)
print('The median location in %s was in %s' % (whichyear, name_median))
m = folium.Map(location=[float(location_median_geo.raw.get('lat')),
                         float(location_median_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_median_geo.raw.get('lat')),
                              float(location_median_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_median).add_to(m)
m

{'place_id': 111237126, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 25678439, 'lat': '46.93519445', 'lon': '7.429954042480473', 'display_name': 'Beachcenter Bern, Südbahnhofstrasse, Weissenbühl, Stadtteil III, Bern, Verwaltungskreis Bern-Mittelland, Verwaltungsregion Bern-Mittelland, Bern/Berne, 3007, Schweiz/Suisse/Svizzera/Svizra', 'address': {'leisure': 'Beachcenter Bern', 'road': 'Südbahnhofstrasse', 'quarter': 'Weissenbühl', 'city_district': 'Stadtteil III', 'city': 'Bern', 'county': 'Verwaltungskreis Bern-Mittelland', 'state_district': 'Verwaltungsregion Bern-Mittelland', 'state': 'Bern/Berne', 'ISO3166-2-lvl4': 'CH-BE', 'postcode': '3007', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['46.9349987', '46.9353902', '7.4296711', '7.430237']}
The median location in 2022 was in Verwaltungskreis Bern-Mittelland


In [25]:
## Get day of location, based on 
# https://stackoverflow.com/a/53979441/323100
# and 
# https://strftime.org
# print(locations[locations.eq(location_north[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d'))

In [26]:
location_north_geo = geolocator.reverse(location_north)
if debug:
    print(location_north_geo.raw)
for p in potentialplacename:
    if location_north_geo.raw.get('address').get(p):
        name_north = location_north_geo.raw.get('address').get(p)
print('The northmost location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_north,
                        location_north_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_north[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_north_geo.raw.get('lat')),
                         float(location_north_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_north_geo.raw.get('lat')),
                              float(location_north_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_north).add_to(m)
m

{'place_id': 268102095, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 836754180, 'lat': '47.56950688734801', 'lon': '7.626694523050589', 'display_name': 'Bäumlihofstrasse, Bäumlihof, Riehen, Basel-Stadt, 4125, Schweiz/Suisse/Svizzera/Svizra', 'address': {'road': 'Bäumlihofstrasse', 'hamlet': 'Bäumlihof', 'town': 'Riehen', 'state': 'Basel-Stadt', 'ISO3166-2-lvl4': 'CH-BS', 'postcode': '4125', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['47.569235', '47.5695504', '7.6261223', '7.626786']}
The northmost location in 2022 was in Bäumlihof, Schweiz/Suisse/Svizzera/Svizra on May, 21


/tmp/ipykernel_213840/381974690.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  locations[locations.eq(location_north[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))


In [27]:
location_east_geo = geolocator.reverse(location_east)
if debug:
    print(location_east_geo.raw)
for p in potentialplacename:
     if location_east_geo.raw.get('address').get(p):
        name_east = location_east_geo.raw.get('address').get(p)
print('The most eastern location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_east,
                        location_east_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_east[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_east_geo.raw.get('lat')),
                         float(location_east_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_east_geo.raw.get('lat')),
                              float(location_east_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_east).add_to(m)
m

{'place_id': 297461494, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 47181, 'lat': '46.4228762', 'lon': '11.7207601', 'display_name': 'San Giovanni di Fassa - Sèn Jan, Comun General de Fascia, Provincia di Trento, Trentino-Alto Adige/Südtirol, Italia', 'address': {'municipality': 'San Giovanni di Fassa - Sèn Jan', 'county': 'Provincia di Trento', 'ISO3166-2-lvl6': 'IT-TN', 'state': 'Trentino-Alto Adige/Südtirol', 'ISO3166-2-lvl4': 'IT-32', 'country': 'Italia', 'country_code': 'it'}, 'boundingbox': ['46.3784359', '46.4759402', '11.5930231', '11.8512924']}
The most eastern location in 2022 was in San Giovanni di Fassa - Sèn Jan, Italia on May, 8


/tmp/ipykernel_213840/2601345277.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  locations[locations.eq(location_east[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))


In [28]:
name_east

'San Giovanni di Fassa - Sèn Jan'

In [29]:
location_south_geo = geolocator.reverse(location_south)
if debug:
    print(location_south_geo.raw)
for p in potentialplacename:
    if location_south_geo.raw.get('address').get(p):
        name_south = location_south_geo.raw.get('address').get(p)
print('The southmost location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_south,
                        location_south_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_south[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_south_geo.raw.get('lat')),
                         float(location_south_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_south_geo.raw.get('lat')),
                              float(location_south_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_south).add_to(m)
m

{'place_id': 297476510, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 40281, 'lat': '40.0933379', 'lon': '9.5080494', 'display_name': 'Orthullè/Urzulei, Ogliastra, Sardigna/Sardegna, 08040, Italia', 'address': {'village': 'Orthullè/Urzulei', 'province': 'Ogliastra', 'ISO3166-2-lvl6': 'IT-OG', 'state': 'Sardigna/Sardegna', 'ISO3166-2-lvl4': 'IT-88', 'postcode': '08040', 'country': 'Italia', 'country_code': 'it'}, 'boundingbox': ['40.0463179', '40.2186146', '9.4368784', '9.5767001']}
The southmost location in 2022 was in Orthullè/Urzulei, Italia on September, 10


/tmp/ipykernel_213840/3516057741.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  locations[locations.eq(location_south[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))


In [30]:
location_west_geo = geolocator.reverse(location_west)
if debug:
    print(location_west_geo.raw)
for p in potentialplacename:
    if location_west_geo.raw.get('address').get(p):
        name_west = location_west_geo.raw.get('address').get(p)
print('The most western location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_west,
                        location_west_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_west[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_west_geo.raw.get('lat')),
                         float(location_west_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_west_geo.raw.get('lat')),
                              float(location_west_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_west).add_to(m)
m

{'place_id': 113843156, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 32654468, 'lat': '46.59990440763477', 'lon': '6.553698191364292', 'display_name': 'A1;A9, Bournens, District du Gros-de-Vaud, Vaud, 1035, Schweiz/Suisse/Svizzera/Svizra', 'address': {'road': 'A1;A9', 'village': 'Bournens', 'county': 'District du Gros-de-Vaud', 'state': 'Vaud', 'ISO3166-2-lvl4': 'CH-VD', 'postcode': '1035', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['46.5971667', '46.6035472', '6.5536687', '6.5540731']}
The most western location in 2022 was in District du Gros-de-Vaud, Schweiz/Suisse/Svizzera/Svizra on April, 21


/tmp/ipykernel_213840/3249128656.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  locations[locations.eq(location_west[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))


In [31]:
location_top_geo = geolocator.reverse(location_top)
if debug:
    print(location_top_geo.raw)
for p in potentialplacename:
    if location_top_geo.raw.get('address').get(p):
        name_top = location_top_geo.raw.get('address').get(p)
print('The highest location in %s was in '
      '%s, %s on %s' % (whichyear,
                        name_top,
                        location_top_geo.raw.get('address').get('country'),
                        locations[locations.eq(location_top[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))
m = folium.Map(location=[float(location_top_geo.raw.get('lat')),
                         float(location_top_geo.raw.get('lon'))],
               tiles=tileprovider,
               zoom_start=zoom_start*2)
folium.CircleMarker(location=[float(location_top_geo.raw.get('lat')),
                              float(location_top_geo.raw.get('lon'))],
                    radius=radius,
                    popup=name_top).add_to(m)
m

{'place_id': 292864543, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 1036016462, 'lat': '45.9373465', 'lon': '7.7292823', 'display_name': 'Matterhorn Glacier Paradise, Zermatt, Visp, Valais/Wallis, 3920, Schweiz/Suisse/Svizzera/Svizra', 'address': {'road': 'Matterhorn Glacier Paradise', 'town': 'Zermatt', 'county': 'Visp', 'state': 'Valais/Wallis', 'ISO3166-2-lvl4': 'CH-VS', 'postcode': '3920', 'country': 'Schweiz/Suisse/Svizzera/Svizra', 'country_code': 'ch'}, 'boundingbox': ['45.9373465', '45.9373845', '7.7292823', '7.7294473']}
The highest location in 2022 was in Visp, Schweiz/Suisse/Svizzera/Svizra on December, 18


/tmp/ipykernel_213840/3728374390.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  locations[locations.eq(location_top[0]).any(1)]['Date'].tolist()[0].strftime('%B, %-d')))


In [32]:
# Show the extreme values in the overview below or not
showextremes = True

In [33]:
# Plot *all* locations on a single map
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
if showextremes:
    # Extreme locations
    folium.CircleMarker(location=location_average,
                        radius=radius,
                        popup='Average location in %s' % name_average).add_to(m)
    folium.CircleMarker(location=location_north,
                        radius=radius,
                        popup='Northmost location in %s' % name_north).add_to(m)
    folium.CircleMarker(location=location_east,
                        radius=radius,
                        popup='Northmost location in %s' % name_east).add_to(m)
    folium.CircleMarker(location=location_south,
                        radius=radius,
                        popup='Northmost location in %s' % name_south).add_to(m)
    folium.CircleMarker(location=location_west,
                        radius=radius,
                        popup='Northmost location in %s' % name_west).add_to(m)
    folium.CircleMarker(location=location_top,
                        radius=radius,
                        popup='Highest location (%s m) on the %s' % (locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0],
                                                                     name_top)).add_to(m)
# All locations, in different ways
singlepoints = False
fast = True
if singlepoints:
    for c, loc in locations.iterrows():
        # not every point, but every x-th one, or else the map is too slow
        if not c % 10:
            folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                radius=2,
                                popup='%s@%s' % (loc.Date, loc.Time),
                                color='darkred'
                               ).add_to(m)
else:
    if fast:
        # FastMarkerCluster
        m.add_child(folium.plugins.FastMarkerCluster(locations[['Latitude', 'Longitude']].values.tolist()))
    else:
        # Markercluster
        mc = folium.plugins.MarkerCluster()
        for c, loc in locations.iterrows():
            mc.add_child(folium.CircleMarker(location=[loc.Latitude, loc.Longitude],
                                             popup='%s@%s' % (loc.Date, loc.Time)))
        m.add_child(mc)
m.save('map-points.html')
m

In [34]:
# Viridis colormap from here: https://www.thedataschool.co.uk/gwilym-lockwood/viridis-colours-tableau/
gradient={0.00: '#440154FF',
          0.05: '#481567FF',
          0.10: '#482677FF',
          0.15: '#453781FF',
          0.20: '#404788FF',
          0.25: '#39568CFF',
          0.30: '#33638DFF',
          0.35: '#2D708EFF',
          0.40: '#287D8EFF',
          0.45: '#238A8DFF',
          0.50: '#1F968BFF',
          0.55: '#20A387FF',
          0.60: '#29AF7FFF',
          0.65: '#3CBB75FF',
          0.70: '#55C667FF',
          0.75: '#73D055FF',
          0.80: '#95D840FF',
          0.85: '#B8DE29FF',
          0.90: '#DCE319FF',
          0.95: '#FDE725FF'}

In [35]:
# Show a heatmap instead of single points
m = folium.Map(location=[locations['Latitude'].mean(),
                         locations['Longitude'].mean()],
               tiles=tileprovider,
               zoom_start=zoom_start)
showextremes=False
if showextremes:
    # Extreme locations
    folium.CircleMarker(location=location_average,
                        radius=radius,
                        popup='Average location in %s' % name_average,
                       ).add_to(m)
    folium.CircleMarker(location=location_north,
                        radius=radius,
                        popup='Northmost location in %s' % name_north,
                       ).add_to(m)
    folium.CircleMarker(location=location_east,
                        radius=radius,
                        popup='Most eastern location in %s' % name_east,
                       ).add_to(m)
    folium.CircleMarker(location=location_south,
                        radius=radius,
                        popup='Southmost location in %s' % name_south,
                       ).add_to(m)
    folium.CircleMarker(location=location_west,
                        radius=radius,
                        popup='Most western location in %s' % name_west,
                       ).add_to(m)
    folium.CircleMarker(location=location_top,
                        radius=radius,
                        popup='Highest location (%s m) on the %s' % (locations[locations['Altitude'] == locations['Altitude'].max()].Altitude.values[0],
                                                                     name_top),
                       ).add_to(m)
# Add heatmap
folium.plugins.HeatMap(locations[['Latitude', 'Longitude']].values.tolist(),
                       gradient=gradient).add_to(m)
m.save('map-heat.html')
m